# Fine-tuning Lag-Llama

In [1]:
!git clone https://github.com/time-series-foundation-models/lag-llama/

Cloning into 'lag-llama'...
remote: Enumerating objects: 294, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 294 (delta 102), reused 86 (delta 77), pack-reused 162
Receiving objects: 100% (294/294), 224.13 KiB | 7.73 MiB/s, done.
Resolving deltas: 100% (143/143), done.


In [2]:
cd /content/lag-llama

/content/lag-llama


In [3]:
!pip install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.1/296.1 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 48.7 MB/s eta 0:00:00
ERROR: pip's dependency resol

In [4]:
!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir /content/lag-llama

lag-llama.ckpt: 100% 29.5M/29.5M [00:00<00:00, 62.8MB/s]
Download complete. Moving file to /content/lag-llama/lag-llama.ckpt
/content/lag-llama/lag-llama.ckpt


In [5]:
!pip install gluonts==0.14.4

In [6]:
from itertools import islice

from matplotlib import pyplot as plt

from tqdm.autonotebook import tqdm

import torch

from gluonts.dataset.pandas import PandasDataset

import pandas as pd

from lag_llama.gluon.estimator import LagLlamaEstimator

<ipython-input-6-eaf45c051a85>:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [7]:
def get_lag_llama_predictions(dataset,
                              prediction_length,
                              context_length=32,
                              device="cuda",
                              num_samples=100,
                              predictor=None):

    ckpt = torch.load("lag-llama.ckpt", map_location=device)
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        num_parallel_samples=num_samples,
    )

    if predictor is None:
      lightning_module = estimator.create_lightning_module()
      transformation = estimator.create_transformation()
      predictor = estimator.create_predictor(transformation, lightning_module)

    forecasts = predictor.predict(
        dataset=dataset
    )

    forecasts = list(forecasts)

    return forecasts

In [8]:
device = 'cuda'
prediction_length = 8
context_length = 64

ckpt = torch.load("lag-llama.ckpt", map_location=device)
estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,

        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        time_feat=estimator_args["time_feat"],

        # Fine-tuning arguments
        nonnegative_pred_samples=True,
        aug_prob=0,
        lr=5e-4,
        batch_size=64,
        num_parallel_samples=20,
        trainer_kwargs = {"max_epochs": 50,}
    )

In [9]:
url = 'https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/main/data/walmart_sales_small.csv'

df = pd.read_csv(url)
df = df[df['Store'] == 1]

input_df = df[:-8]
test_df = df[-8:]

input_df['Weekly_Sales'] = input_df['Weekly_Sales'].astype('float32')

<ipython-input-9-abc609e88630>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['Weekly_Sales'] = input_df['Weekly_Sales'].astype('float32')


In [10]:
input_ds = PandasDataset.from_long_dataframe(input_df, target='Weekly_Sales', item_id='Store')

In [11]:
predictor = estimator.train(input_ds, cache_data=True, shuffle_buffer_length=1000)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name          | Type               | Params
-----------------------------------------------------
0 | model         | LagLlamaModel      | 2.4 M 
1 | augmentations | ApplyAugmentat

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 13.86050 (best 13.86050), saving model to '/content/lag-llama/lightning_logs/version_0/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 50: 'train_loss' reached 13.86050 (best 13.86050), saving model to '/content/lag-llama/lightning_logs/version_0/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 13.10804 (best 13.10804), saving model to '/content/lag-llama/lightning_logs/version_0/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 1, global step 100: 'train_loss' reached 13.10804 (best 13.10804), saving model to '/content/lag-llama/lightning_logs/version_0/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 13.00644 (best 13.00644), saving model to '/content/lag-llama/lightning_logs/version_0/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO:light

In [12]:
finetune_forecasts = get_lag_llama_predictions(
        dataset=input_ds,
        prediction_length=prediction_length,
        predictor=predictor,
    )

In [13]:
finetune_forecasts[0].samples.shape

(20, 8)

In [14]:
import numpy as np

def get_median_and_ci(data,
                      start_date,
                      horizon,
                      freq,
                      id,
                      confidence=0.95):

    n_samples, n_timesteps = data.shape

    # Calculate the median for each timestep
    medians = np.median(data, axis=0)

    # Calculate the lower and upper percentile for the given confidence interval
    lower_percentile = (1 - confidence) / 2 * 100
    upper_percentile = (1 + confidence) / 2 * 100

    # Calculate the lower and upper bounds for each timestep
    lower_bounds = np.percentile(data, lower_percentile, axis=0)
    upper_bounds = np.percentile(data, upper_percentile, axis=0)

    pred_dates = pd.date_range(start=start_date, periods=horizon, freq=freq)
    formatted_dates = pred_dates.strftime('%m-%d-%Y').tolist()

    # Create a DataFrame with the results
    df = pd.DataFrame({
        'Date': formatted_dates,
        'Store': id,
        'Lag-Llama': medians,
        f'Lag-Llama-lo-{int(confidence*100)}': lower_bounds,
        f'Lag-Llama-hi-{int(confidence*100)}': upper_bounds
    })

    return df

In [15]:
finetuned_preds = get_median_and_ci(finetune_forecasts[0].samples,
                                    start_date='2012-09-07',
                                    horizon=8,
                                    freq='W-FRI',
                                    id=1,
                                    confidence=0.80)

finetuned_preds

,Date,Store,Lag-Llama,Lag-Llama-lo-80,Lag-Llama-hi-80
0,09-07-2012,1,1646512.250,1.641362e+06,1.657577e+06
1,09-14-2012,1,1616172.500,1.608035e+06,1.621749e+06
2,09-21-2012,1,1516381.000,1.510720e+06,1.520407e+06
3,09-28-2012,1,1566029.750,1.560994e+06,1.572415e+06
4,10-05-2012,1,1755081.125,1.750029e+06,1.761098e+06
5,10-12-2012,1,1628259.750,1.621195e+06,1.632376e+06
6,10-19-2012,1,1501428.000,1.497410e+06,1.509792e+06
7,10-26-2012,1,1569666.750,1.562812e+06,1.574566e+06


In [16]:
def calculate_mae_smape(pred_df, test_df, target_col, pred_col):

    # Extract the relevant columns
    y_true = test_df[target_col].values
    y_pred = pred_df[pred_col].values

    # Calculate MAE
    mae = int(np.mean(np.abs(y_true - y_pred)))

    # Calculate sMAPE
    denominator = np.abs(y_true) + np.abs(y_pred)
    smape = round(np.mean(2.0 * np.abs(y_true - y_pred) / denominator) * 100,2)

    return mae, smape

In [17]:
mae_finetuned, smape_finetuned = calculate_mae_smape(finetuned_preds, test_df, 'Weekly_Sales', 'Lag-Llama')

print(mae_finetuned, smape_finetuned)

59419 3.78
